In [22]:
# Ejemplo de crear una api con flask

# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LogisticRegression

import pickle
import joblib

import warnings
warnings.filterwarnings('ignore')


In [23]:
# Importamos el dataset del Titanic
df = sns.load_dataset("titanic")
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [24]:
# Conservamos sólo sexo, edad y clase
df = df[['sex', 'age', 'class', 'survived']]
df

,sex,age,class,survived
0,male,22.0,Third,0
1,female,38.0,First,1
2,female,26.0,Third,1
3,female,35.0,First,1
4,male,35.0,Third,0
...,...,...,...,...
886,male,27.0,Second,0
887,female,19.0,First,1
888,female,NaN,Third,0
889,male,26.0,First,1


In [25]:
# Dummyficamos la variable sexo y clase
df = pd.get_dummies(df)
df = df.dropna()
df

,age,survived,sex_female,sex_male,class_First,class_Second,class_Third
0,22.0,0,False,True,False,False,True
1,38.0,1,True,False,True,False,False
2,26.0,1,True,False,False,False,True
3,35.0,1,True,False,True,False,False
4,35.0,0,False,True,False,False,True
...,...,...,...,...,...,...,...
885,39.0,0,True,False,False,False,True
886,27.0,0,False,True,False,True,False
887,19.0,1,True,False,True,False,False
889,26.0,1,False,True,True,False,False


In [26]:
# Aplicamos el modelo de regresión logística para determinar si sobrevive o no
X = df.drop(columns = 'survived')
y = df['survived']

model = LogisticRegression()
model.fit(X, y)
model.score(X, y)

0.7899159663865546

In [27]:
# nuevo_txt = open('prueba1.txt', 'w')
# nuevo_txt.write('Hola mundo 10, hola mundo 11')
# nuevo_txt.close()


In [28]:
# Guardamos el modelo con pickle y con joblib

pickle.dump(model, open('model1.pkl', 'wb')) # Guardamos el modelo en un archivo pickle, wb = write binary

joblib.dump(model, 'model1.joblib')


['model1.joblib']

In [29]:
# Cargo el modelo
modelp = pickle.load(open('model1.pkl', 'rb')) # rb = read binary
modelj = joblib.load('model1.joblib')

In [30]:
# Hago una predicción
pp = modelp.predict([[30, 0, 1, 0, 1, 0]])
pj = modelj.predict([[30, 0, 1, 0, 1, 0]])

print(pp)
print(pj)

[0]
[0]


In [31]:
# Creamos un diccionario tipo json con los datos ingresados por el usuario
# Asignamos los valores ingresados por el usuario
def prediccion(pasajero):
    # Creamos un array de 0 con la cantidad de columnas del dataset
    nuevo = np.zeros(6)

    # El primer valor del array es la edad
    nuevo[0] = pasajero['edad']
    # El segundo valor del array es el sexo_m y el tercero el sexo_f
    if pasajero['sexo'] == 0:
        nuevo[1] = 1
    else:
        nuevo[2] = 1
    # El cuarto valor del array es la clase_1, el quinto la clase_2 y el sexto la clase_3
    if pasajero['clase'] == 1:
        nuevo[3] = 1
    elif pasajero['clase'] == 2:
        nuevo[4] = 1
    else:
        nuevo[5] = 1

    # Creamos un dataframe con el array
    nuevo = pd.DataFrame(nuevo.reshape(1, -1), columns = X.columns)

    # Hacemos la predicción
    respuesta = modelp.predict(nuevo)

    if respuesta[0] == 0:
        mensaje = 'No sobrevive'
    else:
        mensaje = 'Sobrevive'
    # print(mensaje)
    
    return mensaje


In [32]:
pasajero = {
    'edad': 30,
    'sexo': 0,
    'clase': 3
}

prediccion(pasajero)

'Sobrevive'